In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gurobipy as gb
import os
from pickle import dump, load
from time import process_time
from copy import deepcopy
from source import*

path = os.getcwd()

In [2]:
file = open(path+"/Results/Configurations/Open Stations/open_stations_18","rb"); S = load(file); file.close()

K_s = {}                # {scenario:{station:[vehicles]}}
K = {}                  # {scenatio:[vehicles]}

for sc in range(25):
    file = open(path+f"/Data/K/K_sc{sc}", "rb")
    K[sc] = load(file); file.close()

    file = open(path+f"/Data/K_s10/Ks_sc{sc}", "rb")
    K_s[sc] = load(file); file.close()


original_stress = {}
for s in S:
    original_stress[s] = np.mean(np.array([len(K_s[sc][s]) for sc in range(25)]))

max_s = max(original_stress, key=original_stress.get)
max_s, original_stress[max_s]

(459, 197.0)

In [3]:
def generate_routes_given_number_chargers(s,S,y,K,K_s_prime,exhaust=False):
    total_serviced_costumers:dict[list[list]] = dict()

    for sc in range(25):
        routes = list()
        K, _, S_k, a, t = load_pickle(path+"/Data/",sc)
        K_s = K_s_prime[sc]
        S_k = {k:[ss for ss in S_k[k] if ss in S] for k in K_s[s]}
        a.update({("s",s):0}); a.update({("e",s):30})
        t.update({("s",s):0}); t.update({("e",s):0})

        mp = gb.Model("Restricted Master Problem")

        dummy_0 = {k:mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=-len(K_s[s]), name=f"st0_{k}") for k in K_s[s]}
        aux = mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=-2, name=f"aux_{s}")

        vehic_assign = {}
        for k in K_s[s]:
            if len(S_k[k]) == 1: sense = gb.GRB.EQUAL
            else: sense = gb.GRB.LESS_EQUAL
            vehic_assign[k] = mp.addLConstr(lhs=dummy_0[k], sense=sense, rhs=1, name=f"V{k}_assignment")

        st_conv = mp.addConstr(aux <= y, f"S{s}_convexity")

        mp.setParam("OutputFlag",0)
        mp.setAttr("ModelSense", -1)

        time0 = process_time(); i = 0
        lbd = []

        while True:
            
            mp.update()
            mp.optimize()

            pi = {k:vehic_assign[k].getAttr("Pi") for k in K_s[s]}
            sigma = st_conv.getAttr("Pi")

            #print(f"\t\tIteration {i}:\t\tMP obj: {round(mp.getObjective().getValue(),2)}\ttime: {round(process_time()-time0,2)}s")
            i += 1

            V,A,rc = get_graph_chargers(s,K_s[s],a,pi,sigma)
            ext = vertices_extensions(V,A)

            routes_DFS = []; covered_nodes = set()
            label_DFS_chargers(v="s",rP=0,tP=0,qP=0,P=[],cK=covered_nodes,L=routes_DFS,s=s,r=rc,t=t,a=a,ext=ext)
            opt = len(routes_DFS)

            for l in routes_DFS:
                col = {k:1 if k in l else 0 for k in K_s[s]}
                new_Col = gb.Column(col.values(),vehic_assign.values())
                lbd.append(mp.addVar(vtype=gb.GRB.CONTINUOUS,obj=sum(col.values()),ub=1,column=new_Col))

                mp.chgCoeff(st_conv,lbd[-1],1)

            if opt == 0: mpsol = mp.getObjective().getValue(); break
        
        for v in mp.getVars():
            v.vtype = gb.GRB.BINARY
        if mpsol < len(K_s[s]): mp.setParam("MIPGap",0.01)
        mp.setParam("MIPFocus",2)
        mp.update(); mp.optimize()

        # print(f"IMP solution: {mp.getObjective().getValue()} assigned vehicles with {y} chargers")
        for l in lbd:
            if l.X > 0.5:
                col = [k for k in K_s[s] if mp.getCoeff(vehic_assign[k],l) == 1]
                routes.append(col)
        

        total_serviced_costumers[sc]:list = list()

        for route in routes:
            total_serviced_costumers[sc].extend(route)
            
        if len(total_serviced_costumers) < len(K_s_prime[sc][s]) and y < 8:
                print(f'{y} number of chargers is infeasible')
                return None
        #break

    print(f"IMP solution: {mp.getObjective().getValue()} assigned vehicles with {y} charger(s)")
    return total_serviced_costumers

In [4]:
ordered_stations:list = sorted(original_stress, key = original_stress.get, reverse = True)     # Get list of stations ordered by stress
number_of_chargers:dict = dict()
stress = deepcopy(original_stress)

K_prime = deepcopy(K)
K_s_prime = deepcopy(K_s) 
# S_k_prime = deepcopy(S_k)

cont:int = 0

while True:
    cont += 1 
    s = ordered_stations[0]
    print(f'--------- Station {s} - {stress[s]} - {original_stress[s]} ---------')
    y:int = 1       # Start giving one charger to the station

    # Iterative process to find number of chargers 
    while True:
        total_serviced_costumers:dict[list[list]] or None = generate_routes_given_number_chargers(s, S, y, K_prime, K_s_prime, False)     # Get the routes for the stations
        
        if total_serviced_costumers == None:  # Station doesn't service all possible costumers in at least one scenario
            y += 1
        else:               # Number of chargers is fixed
            break
    
    # Set number of chargers and remove already serviced vehicles
    number_of_chargers[s] = y
    print(f'--------- Station {s} set to {number_of_chargers[s]} chargers\n')

    for u in range(25):
        for vehicle in total_serviced_costumers[u]:
            K_prime[u].remove(vehicle)
        
            for ss in S:
                if vehicle in K_s_prime[u][ss]:
                    K_s_prime[u][ss].remove(vehicle)

    S.remove(s)
    stress:dict = {}
    for s in S:
        stress[s] = np.mean(np.array([len(K_s_prime[sc][s]) for sc in range(25)]))    
    ordered_stations:list = sorted(stress, key = stress.get, reverse = True)     # Get list of stations ordered by stress

    if cont == 600 or sum(len(K_s_prime[sc][s]) for sc in range(25) for s in S)==0:
        break


print(number_of_chargers)

--------- Station 459 - 197.0 - 197.0 ---------
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-11
1 number of chargers is infeasible
2 number of chargers is infeasible
3 number of chargers is infeasible
4 number of chargers is infeasible
5 number of chargers is infeasible
6 number of chargers is infeasible
7 number of chargers is infeasible
IMP solution: 145.0 assigned vehicles with 8 charger(s)
--------- Station 459 set to 8 chargers

--------- Station 379 - 196.12 - 196.12 ---------
1 number of chargers is infeasible
2 number of chargers is infeasible
3 number of chargers is infeasible
4 number of chargers is infeasible
5 number of chargers is infeasible
6 number of chargers is infeasible
7 number of chargers is infeasible
IMP solution: 148.0 assigned vehicles with 8 charger(s)
--------- Station 379 set to 8 chargers

--------- Station 1154 - 169.0 - 169.0 ---------
1 number of chargers is infeasible
2 number of chargers is infeasible
3 number

In [4]:
routes_DFS = number_of_chargers(max_s,S,8)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-08
		Iteration 0:		MP obj: 0.0	time: 0.0s
		Iteration 1:		MP obj: 141.0	time: 5.56s
		Iteration 2:		MP obj: 141.0	time: 11.3s
		Iteration 3:		MP obj: 141.0	time: 16.48s
		Iteration 4:		MP obj: 141.0	time: 21.67s
		Iteration 5:		MP obj: 141.0	time: 26.44s
		Iteration 6:		MP obj: 141.0	time: 30.94s
		Iteration 7:		MP obj: 141.0	time: 35.08s
		Iteration 8:		MP obj: 141.0	time: 39.69s
		Iteration 9:		MP obj: 141.0	time: 44.97s
		Iteration 10:		MP obj: 141.0	time: 49.41s
		Iteration 11:		MP obj: 141.11	time: 54.58s
		Iteration 12:		MP obj: 141.13	time: 59.44s
		Iteration 13:		MP obj: 141.26	time: 64.47s
		Iteration 14:		MP obj: 141.31	time: 69.19s
		Iteration 15:		MP obj: 141.36	time: 73.69s
		Iteration 16:		MP obj: 141.43	time: 78.75s
		Iteration 17:		MP obj: 141.49	time: 83.95s
		Iteration 18:		MP obj: 141.5	time: 88.47s
		Iteration 19:		MP obj: 141.54	time: 94.67s
		Iteration 20:		MP obj: 141.58	time:

In [ ]:
from multiprocess import pool

def generate_routes_given_number_chargers(s,S,y,K,K_s_prime,exhaust=False):
    total_serviced_costumers:dict[list[list]] = dict()

    def myFunction(sc):
        routes = list()
        K, _, S_k, a, t = load_pickle(path+"/Data/",sc)
        K_s = K_s_prime[sc]
        S_k = {k:[ss for ss in S_k[k] if ss in S] for k in K_s[s]}
        a.update({("s",s):0}); a.update({("e",s):30})
        t.update({("s",s):0}); t.update({("e",s):0})

        mp = gb.Model("Restricted Master Problem")

        dummy_0 = {k:mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=-len(K_s[s]), name=f"st0_{k}") for k in K_s[s]}
        aux = mp.addVar(vtype=gb.GRB.CONTINUOUS, obj=-2, name=f"aux_{s}")

        vehic_assign = {}
        for k in K_s[s]:
            if len(S_k[k]) == 1: sense = gb.GRB.EQUAL
            else: sense = gb.GRB.LESS_EQUAL
            vehic_assign[k] = mp.addLConstr(lhs=dummy_0[k], sense=sense, rhs=1, name=f"V{k}_assignment")

        st_conv = mp.addConstr(aux <= y, f"S{s}_convexity")

        mp.setParam("OutputFlag",0)
        mp.setAttr("ModelSense", -1)

        time0 = process_time(); i = 0
        lbd = []

        while True:
            
            mp.update()
            mp.optimize()

            pi = {k:vehic_assign[k].getAttr("Pi") for k in K_s[s]}
            sigma = st_conv.getAttr("Pi")

            #print(f"\t\tIteration {i}:\t\tMP obj: {round(mp.getObjective().getValue(),2)}\ttime: {round(process_time()-time0,2)}s")
            i += 1

            V,A,rc = get_graph_chargers(s,K_s[s],a,pi,sigma)
            ext = vertices_extensions(V,A)

            routes_DFS = []; covered_nodes = set()
            label_DFS_chargers(v="s",rP=0,tP=0,qP=0,P=[],cK=covered_nodes,L=routes_DFS,s=s,r=rc,t=t,a=a,ext=ext)
            opt = len(routes_DFS)

            for l in routes_DFS:
                col = {k:1 if k in l else 0 for k in K_s[s]}
                new_Col = gb.Column(col.values(),vehic_assign.values())
                lbd.append(mp.addVar(vtype=gb.GRB.CONTINUOUS,obj=sum(col.values()),ub=1,column=new_Col))

                mp.chgCoeff(st_conv,lbd[-1],1)

            if opt == 0: mpsol = mp.getObjective().getValue(); break
        
        for v in mp.getVars():
            v.vtype = gb.GRB.BINARY
        if mpsol < len(K_s[s]): mp.setParam("MIPGap",0.01)
        mp.setParam("MIPFocus",2)
        mp.update(); mp.optimize()

        # print(f"IMP solution: {mp.getObjective().getValue()} assigned vehicles with {y} chargers")
        for l in lbd:
            if l.X > 0.5:
                col = [k for k in K_s[s] if mp.getCoeff(vehic_assign[k],l) == 1]
                routes.append(col)
        

        total_serviced_costumers[sc]:list = list()

        for route in routes:
            total_serviced_costumers[sc].extend(route)
            
        if len(total_serviced_costumers) < len(K_s_prime[sc][s]) and y < 8:
                print(f'{y} number of chargers is infeasible')
                return None
        #break

        print(f"IMP solution: {mp.getObjective().getValue()} assigned vehicles with {y} charger(s)")
        return total_serviced_costumers
    
    lab = pool.Pool()
    Results = lab.map(myFunction, list(range(25)))

In [5]:
print(f"DFS Labeling covered vehicles: {sum(len(r) for r in routes_DFS[0])}")

DFS Labeling covered vehicles: 141


In [8]:
print(*[len(i) for i in routes_DFS[0]])

28 22 19 17 16 14 14 11
